In [1]:
%matplotlib inline

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mido import MidiFile, tick2second
from pretty_midi import PrettyMIDI
import pickle
import joblib
import subprocess
import os
import os.path
import random
from ExtractBootlegFeatures1 import *
import multiprocessing
import time

# Extract Bootleg Features from Images

In [4]:
def convert_Pdf_wrapper(i):
    basedir = 'experiments/allBootlegScores-new/'
    outdir = '{}/scores/'.format(basedir)
    imagedir = '/data1/dyang/Datasets/5000dataset/images1/'
    numPdfs=5024;
    if i >= numPdfs:
        return
    if not os.path.isdir(outdir):
        os.makedirs(outdir)
    
    outfile = outdir + 'p{}.pkl'.format(str(i))
    if i <= 200:
        filepath = imagedir + 'p'+str(i)+'/'
    else:
        filepath = imagedir + 'p'+str(i)+'/'
    if os.path.exists(outfile):
        print(str(i)+" is already there")
        return
    total_bscore = np.zeros((62,1))
    lst = 1 #len(os.listdir(filepath))
    if lst == 1:
        if i<=200:
            imagepath = '20210726165646.jpg' #'p{}.jpg'.format(i)
        else:
            imagepath = filepath+'p{}.jpg'.format(i)
        try:
            bscore_query = processImageFile(imagepath,None)
        except:
            bscore_query = np.zeros((62,1))
        total_bscore = np.concatenate((total_bscore, bscore_query),axis=1)
    else:
        for count in range(lst):
            try:
                if i<=200:
                    imagepath = filepath+'p{}-{}.jpg'.format(i,count)
                else:
                    imagepath = filepath+'p{}-{}.jpg'.format(i,count)
                bscore_query=processImageFile(imagepath,None)
            except:
                bscore_query = np.zeros((62,1))
            total_bscore = np.concatenate((total_bscore, bscore_query),axis=1)
    with open(outfile, 'wb') as f:
        print("Saving p" + str(i))
        pickle.dump(total_bscore, f)
    print(total_bscore.shape)
        #visualizeLongBootlegScore(total_bscore, [13,15,17,19,21,35,37,39,41,43])

In [4]:
convert_Pdf_wrapper(23)

23 is already there


In [ ]:
inputs = [[i] for i in range(1,4825)]
n_cores = 28

pool = multiprocessing.Pool(processes=n_cores)
outputs = list(pool.starmap(convert_Pdf_wrapper, inputs))

# Construct Reverse Index

In [3]:
piece_to_num = {}
piecedir = "cfg_files/db.list"
with open(piecedir, "r")as f:
    for i, piece in enumerate(f):
        piece = piece.strip().strip('\n')
        piece =piece.split('/')[-1][:-4]
        piece_to_num[i] = piece

In [8]:
with open("piece_to_num.pkl",'wb')as f:
    pickle.dump(piece_to_num,f)

In [2]:
def bootlegHash(arr):
    bitstring = ""
    for i in range(len(arr)):
        if arr[i]==1:
            bitstring+="1"
        else:
            bitstring +="0"
    bitstring = bitstring+"00"
    hashint = int(bitstring, 2)
    hashint = np.uint64(hashint)
    return hashint

In [6]:
def getTotalBscore(bscore_file):
    bscore_array = []
    with open (bscore_file,'rb') as f:
        bscore_array = pickle.load(f)
    f.close()
    total_bscore = np.array([]).reshape(62,0)
    page_array = []
    for page in bscore_array:
        total_page = np.array([]).reshape(62,0)
        for num in page:
            col = np.array(decodeColumn(num)).reshape(62,-1)
            total_page = np.concatenate((total_page,col),axis=1)
        total_bscore = np.concatenate((total_bscore,total_page),axis=1)
        page_array.append(total_page)
    return total_bscore,page_array

In [7]:
def decodeColumn(num):
    col = []
    for i in range(62):
        col.insert(0,num%2)
        num = int(num/2)
    return col

In [8]:
def getRandomdb(size, mode, seed = 1):
    # Generates db.list
    f = open("cfg_files/db.list", "w")
    random.seed(seed)
    randomDir = []
    if(mode == "all"):
        directory = 'experiments/bootleg_output/'
        for root, dirs, files in os.walk(directory):
            for file in files:
                fname = os.path.join(root,file)
                if fname[-4:] == ".pkl":
                    num = fname.split('/')[-1]
                    str2 = 'experiments/bootleg_output/'+num+"\n"
                    f.write(str2)
        f.close()    
        return
    if size > 200:
        randomDir = random.sample(range(201, 36834), size-200)
    print(randomDir)
    for i in range(1,201):
        num = str(i)
        str2 = 'experiments/bootleg_output/'+"p"+num+".pkl"+"\n"
        f.write(str2)
        
    for i in randomDir:
        num = str(i)
        str2 = 'experiments/bootleg_output/'+"p"+num+".pkl"+"\n"
        f.write(str2)
    f.close()

In [7]:
getRandomdb(200,"all")

In [9]:
def Singular_DB(data, rindex):
    for colindex in range(len(data.T)):
        col = data.T[colindex]
        hashint = bootlegHash(col)
        if hashint == 0:
            continue
        pieceStr = curfile.split('/')[-1][:-4]

        if hashint in rindex:
            if pieceStr in rindex[hashint]:
                rindex[hashint][pieceStr].append(colindex)
            else:
                rindex[hashint][pieceStr]=[colindex]
        else:
            rindex[hashint]={}
            rindex[hashint][pieceStr]=[colindex]
    return rindex

In [10]:
def N_Gram_DB(data, rindex, num, N_Gram = 3):
    for colindex in range(len(data)):
        cols = []
        try:
            for i in range(N_Gram):
                cols.append(data[colindex+i])
        except IndexError:
            continue
        fp = []
        equals_Zero = True
        for column in cols:
            hashint = bootlegHash(column)
            fp.append(hashint)
            if hashint != 0:
                equals_Zero = False
        if equals_Zero == True:
            continue
        fp = tuple(fp)
        if fp in rindex:
            if num in rindex[fp]:
                rindex[fp][num].append(colindex)
            else:
                rindex[fp][num]=[colindex]
        else:
            rindex[fp]={}
            rindex[fp][num]=[colindex]
    return rindex

In [11]:
def Dynamic_N_Gram_DB(data, rindex, counts, num, threshold):
    for colindex in range(len(data.T)):
        N_Gram = 1
        cols = []
        while(True):
            try:
                hashint = bootlegHash(data.T[colindex+N_Gram-1])
                if hashint == 0:
                    break
                cols.append(hashint)
            except IndexError:
                break
            fp = tuple(cols)
            numMatches = counts[N_Gram-1][fp]
            if numMatches < threshold or N_Gram == 4:
                if fp in rindex:
                    if num in rindex[fp]:
                        rindex[fp][num].append(colindex)
                    else:
                        rindex[fp][num] = [colindex]
                else:
                    rindex[fp] = {}
                    rindex[fp][num] = [colindex]
                break
            N_Gram+=1
    return rindex

In [4]:
def createCountFile(outfile, rindex):
    rindex_count = {}
    for key in rindex:
        count = 0
        subDict = rindex[key]
        for key1 in subDict:
            count += len(subDict[key1])
        rindex_count[key] = count
    with open (outfile,"wb") as f:
        pickle.dump(rindex_count,f)
    f.close()

In [ ]:
start = time.process_time()
rindex = {}
fpMap = {}
filelist = 'cfg_files\\db.list'
N=1
outfile = 'G:\\experiments\\indices\\N_Gram_{}_ALL.pkl'.format(N)
mode = "N_GRAM"
with open(filelist, 'r') as f:
    failed = []
    for i, curfile in enumerate(f):
        curfile = curfile.strip().strip('\n')
        #print("Processed:", count)
        try:
            num = curfile.split('\\')[-1][0]
            if(num == 'd'):
                data, _ = getTotalBscore(curfile)  #这一步是将整数
            else:
                with open(curfile, 'rb') as pickle_file:
                    data = pickle.load(pickle_file)
            elif mode == "N_GRAM":
                rindex = N_Gram_DB(data, rindex, i, N_Gram=N)   #固定长度长度
            elif mode == "DYNAMIC_N_GRAM":
                rindex = Dynamic_N_Gram_DB(data, rindex, counts, i, threshold)  #动态长度
        except:
            failed.append(curfile)
    print(failed)
with open(outfile,'wb') as f:
    pickle.dump(rindex,f)
if mode == "N_GRAM":
    outfile = os.path.splitext(outfile)[0][:-3]+"COUNT.pkl"
    createCountFile(outfile, rindex)

# 构造db.list

In [6]:
basepath = "bootleg_total/"
filelist = 'cfg_files/db.list'
with open(filelist,"w") as f:
    for root,dirs,files in os.walk(basepath):
        for filename in files:
            f.write(os.path.join(root,filename)+'\n')

In [10]:
filelist = 'cfg_files/pb.list'
with open(filelist,'w') as f:
    for i in range(1,201):
        f.write('bootleg_total/p'+str(i)+'.pkl\n')